In [72]:
!pip install holisticai scikit-learn pandas numpy matplotlib seaborn jax -q

from holisticai.datasets import load_dataset
import numpy as np
from collections import Counter
from holisticai.bias.metrics import average_odds_diff, equal_opportunity_diff
import pandas as pd
from sklearn.metrics import recall_score, confusion_matrix

dataset = load_dataset("compas_two_year_recid", protected_attribute="sex")
split_dataset = dataset.train_test_split(test_size=0.2, random_state=42)
train = split_dataset['train']
test = split_dataset['test']

dataset2 = load_dataset("bank_marketing", protected_attribute="marital")
split_dataset2 = dataset2.train_test_split(test_size=0.2, random_state=42)
train2 = split_dataset2['train']
test2 = split_dataset2['test']

### Pre-Pruning Method

In [89]:
import numpy as np
from collections import Counter
# from holisticai.bias.metrics import average_odds_diff
import pandas as pd
from sklearn.metrics import recall_score, confusion_matrix

class PrePruneDecisionTreeClassifier:
    def __init__(self, max_depth=5, min_samples_split=2, fairness_weight=0.5):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.fairness_weight = fairness_weight
        self.tree_ = None

    def _calculate_fairness_deviation(self, group_a, group_b, y_pred, y_true):
        """Safe calculation of fairness deviation with error handling"""

       # NOTE: the odds diff and equal opportunity diff are 0 a lot
        deviation =  1.0 - np.mean(
            [
                np.abs(average_odds_diff(group_a, group_b, y_pred, y_true)+1e-6),
                np.abs(equal_opportunity_diff(group_a, group_b, y_pred, y_true)+1e-6)
            ]
        )
        return deviation

    def _calculate_information_gain(self, X, y, groups, feature_idx, threshold):
        left_idxs, right_idxs = self._split(X[:, feature_idx], threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0, float('inf')

        parent_entropy = self._entropy(y)
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        # Calculate fairness metrics safely
        group_a_left = groups[left_idxs] == 0
        group_b_left = groups[left_idxs] == 1
        group_a_right = groups[right_idxs] == 0
        group_b_right = groups[right_idxs] == 1

        # Use the most common label in each split as the prediction
        y_pred_left = np.full_like(y[left_idxs], self._most_common_label(y[left_idxs]))
        y_pred_right = np.full_like(y[right_idxs], self._most_common_label(y[right_idxs]))

        # Calculate fairness deviations safely
        fairness_deviation_left = self._calculate_fairness_deviation(
            group_a_left, group_b_left, y_pred_left, y[left_idxs])
        fairness_deviation_right = self._calculate_fairness_deviation(
            group_a_right, group_b_right, y_pred_right, y[right_idxs])

        # Average the fairness deviations
        fairness_deviation = (fairness_deviation_left + fairness_deviation_right) / 2

        information_gain = parent_entropy - child_entropy
        return information_gain, fairness_deviation

    def fit(self, X, y, groups):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        if isinstance(groups, pd.Series):
            groups = groups.values
        self.tree_ = self._grow_tree(X, y, groups, depth=0)

    # Rest of the CustomDecisionTreeClassifier methods remain the same...
    def _grow_tree(self, X, y, groups, depth):
        num_samples, num_features = X.shape
        if (depth >= self.max_depth or num_samples < self.min_samples_split or len(set(y)) == 1):
            return {'label': self._most_common_label(y)}

        best_split = self._find_best_split(X, y, groups)
        if not best_split:
            return {'label': self._most_common_label(y)}

        left_idxs, right_idxs = self._split(X[:, best_split['feature_idx']], best_split['threshold'])

        left_subtree = self._grow_tree(X[left_idxs], y[left_idxs], groups[left_idxs], depth + 1)
        right_subtree = self._grow_tree(X[right_idxs], y[right_idxs], groups[right_idxs], depth + 1)

        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
                'left': left_subtree, 'right': right_subtree}

    def _find_best_split(self, X, y, groups):
        num_samples, num_features = X.shape
        best_split = {}
        best_gain = -float('inf')

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain, fairness_deviation = self._calculate_information_gain(X, y, groups, feature_idx, threshold)
                
                fairness_penalty = self.fairness_weight * fairness_deviation
                print(f"fairness pen: {fairness_penalty}, fairness deviation: {fairness_deviation}")
                adjusted_gain = gain - fairness_penalty

                if adjusted_gain > best_gain:
                    best_gain = adjusted_gain
                    best_split = {'feature_idx': feature_idx, 'threshold': threshold}

        return best_split if best_gain != -float('inf') else None

    def _split(self, feature_column, threshold):
        left_idxs = np.argwhere(feature_column <= threshold).flatten()
        right_idxs = np.argwhere(feature_column > threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        return np.array([self._traverse_tree(x, self.tree_) for x in X])

    def _traverse_tree(self, x, node):
        if 'label' in node:
            return node['label']
        if x[node['feature_idx']] <= node['threshold']:
            return self._traverse_tree(x, node['left'])
        return self._traverse_tree(x, node['right'])

class PrePruneRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, fairness_weight=0.5, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.fairness_weight = fairness_weight
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y, groups):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        if isinstance(groups, pd.Series):
            groups = groups.values

        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        self.trees = []
        for _ in range(self.n_estimators):
            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[idxs], y[idxs]
            group_sample = groups[idxs]

            tree = PrePruneDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                fairness_weight=self.fairness_weight
            )
            tree.fit(X_sample, y_sample, group_sample)
            self.trees.append(tree)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.round(tree_preds.mean(axis=0)).astype(int)
    
# Example usage
X = train['X']
y = train['y']
groups = np.where(train['p_attrs']["sex"].to_numpy() == "Male", 1, 0)
# Create and fit the model
pre_prune_clf = PrePruneRandomForestClassifier(
    n_estimators=5,
    max_depth=16,
    fairness_weight=0.5,
    random_state=42
)
pre_prune_clf.fit(X, y, groups)

# Make predictions
predictions = pre_prune_clf.predict(test['X'])

print("accuracy", np.count_nonzero(predictions == test['y'])/len(test['y']))
print("equal opportunity diff", equal_opportunity_diff(test['group_a'], test['group_b'], predictions, test['y']))
print("average odds diff", average_odds_diff(test['group_a'], test['group_b'], predictions, test['y']))

fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fairness deviation: 0.999999
fairness pen: 0.4999995, fai

In [9]:
print("accuracy", np.count_nonzero(predictions == test['y'])/len(test['y']))
print("equal opportunity diff", equal_opportunity_diff(test['group_a'], test['group_b'], predictions, test['y']))
print("average odds diff", average_odds_diff(test['group_a'], test['group_b'], predictions, test['y']))

accuracy 0.6518218623481782
equal opportunity diff -0.02502725740905931
average odds diff -0.03815068145654987


In [10]:
# 1. why is fairness deviation = 0
# 2. Add terms for race and gender bias (?) + model needs to trade off
# 3. Fix hyperparmaeter optimization to get the best model + scores]


# Limitation: protected characterstic needs to be binary 
# Extension: pairwise bias and how to mitigate that 

### Post-Pruning Method

In [90]:
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.metrics import recall_score, accuracy_score
from holisticai.bias.metrics import average_odds_diff

class PostPruneDecisionTreeClassifier:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        self.tree_ = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        if (depth >= self.max_depth or num_samples < self.min_samples_split or len(set(y)) == 1):
            return {'label': self._most_common_label(y)}

        best_split = self._find_best_split(X, y)
        
        if not best_split:
            return {'label': self._most_common_label(y)}

        left_idxs, right_idxs = self._split(X[:, best_split['feature_idx']], best_split['threshold'])

        # If either split is empty, return the most common label
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return {'label': self._most_common_label(y)}

        left_subtree = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_subtree = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
                'left': left_subtree, 'right': right_subtree}

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_split = {}
        best_gain = -float('inf')

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self._calculate_information_gain(X, y, feature_idx, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_split = {'feature_idx': feature_idx, 'threshold': threshold}

        return best_split if best_gain != -float('inf') else None
    

    def _calculate_fairness_deviation(self, group_a, group_b, y_pred, y_true):
        """Safe calculation of fairness deviation with error handling"""

       # NOTE: the odds diff and equal opportunity diff are 0 a lot
        deviation =  1.0 - np.mean(
            [
                np.abs(average_odds_diff(group_a, group_b, y_pred, y_true)+1e-6),
                np.abs(equal_opportunity_diff(group_a, group_b, y_pred, y_true)+1e-6)
            ]
        )
        return deviation

    def _calculate_information_gain(self, X, y, feature_idx, threshold):
        left_idxs, right_idxs = self._split(X[:, feature_idx], threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        parent_entropy = self._entropy(y)
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, feature_column, threshold):
        left_idxs = np.argwhere(feature_column <= threshold).flatten()
        right_idxs = np.argwhere(feature_column > threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        if len(y) == 0:
            return 0
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        if len(y) == 0:
            return 0  # Default to label 0 if y is empty
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        return np.array([self._traverse_tree(x, self.tree_) for x in X])

    def _traverse_tree(self, x, node):
        if 'label' in node:
            return node['label']
        if x[node['feature_idx']] <= node['threshold']:
            return self._traverse_tree(x, node['left'])
        return self._traverse_tree(x, node['right'])

    def prune_for_fairness_and_accuracy(self, X, y, groups):
        """
        Post-training pruning to improve fairness and maintain accuracy.
        Start pruning from the leaves to avoid removing too much of the tree at once.
        """
        group_a = np.where(groups == 1, 1, 0)
        group_b = np.where(groups == 0, 1, 0)
        initial_leaves_count = self._count_leaves(self.tree_)
        initial_fairness = self._calculate_fairness_deviation(group_a, group_b, self.predict(X), y)#1 - average_odds_diff(group_a, group_b, self.predict(X), y)
        initial_accuracy = accuracy_score(y, self.predict(X))

        self._prune_tree(self.tree_, X, y, groups)

        final_leaves_count = self._count_leaves(self.tree_)
        final_fairness = self._calculate_fairness_deviation(group_a, group_b, self.predict(X), y) #1 - average_odds_diff(groups, groups, self.predict(X), y)
        final_accuracy = accuracy_score(y, self.predict(X))

        print(f"Initial number of leaves: {initial_leaves_count}")
        print(f"Final number of leaves: {final_leaves_count}")
        print(f"Difference in number of leaves: {initial_leaves_count - final_leaves_count}")
        print(f"Fairness before pruning: {initial_fairness}")
        print(f"Fairness after pruning: {final_fairness}")
        print(f"Difference in fairness: {final_fairness - initial_fairness}")
        print(f"Accuracy before pruning: {initial_accuracy}")
        print(f"Accuracy after pruning: {final_accuracy}")
        print(f"Difference in accuracy: {final_accuracy - initial_accuracy}")

    def _prune_tree(self, node, X, y, groups):
        if 'label' in node:
            return

        left_idxs, right_idxs = self._split(X[:, node['feature_idx']], node['threshold'])
        
        # Check if split produces empty nodes
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            node.clear()
            node['label'] = self._most_common_label(y)
            return

        # Recursively prune left and right subtrees first (bottom-up approach)
        self._prune_tree(node['left'], X[left_idxs], y[left_idxs], groups[left_idxs])
        self._prune_tree(node['right'], X[right_idxs], y[right_idxs], groups[right_idxs])

        # Only consider pruning if both children are leaves
        if 'label' not in node['left'] or 'label' not in node['right']:
            return

        # Calculate metrics before pruning
        group_a = np.where(groups == 1, 1, 0)
        group_b = np.where(groups == 0, 1, 0)


        y_pred = self.predict(X)
        accuracy_before = accuracy_score(y, y_pred)

        fairness_before = self._calculate_fairness_deviation(group_a, group_b, y_pred, y)#1 - average_odds_diff(group_a, group_b, y_pred, y)


        # Temporarily prune the node
        original_node = node.copy()
        node.clear()
        node['label'] = self._most_common_label(y)

        # Calculate metrics after pruning
        y_pred_pruned = self.predict(X)
        accuracy_after = accuracy_score(y, y_pred_pruned)
        fairness_after = self._calculate_fairness_deviation(group_a, group_b, y_pred_pruned, y)#1 - average_odds_diff(group_a, group_b, y_pred_pruned, y)

        # Define minimum acceptable changes
        MIN_ACCURACY_DROP = 0.01  # Allow maximum 2% accuracy drop
        MIN_FAIRNESS_IMPROVEMENT = 0.03  # Require at least 5% fairness improvement

        # Restore the node if:
        # 1. Accuracy drops more than threshold OR
        # 2. Fairness doesn't improve enough
        if (accuracy_before - accuracy_after > MIN_ACCURACY_DROP or 
            fairness_after - fairness_before < MIN_FAIRNESS_IMPROVEMENT):
            node.clear()
            node.update(original_node)
        
    def _count_leaves(self, node):
        if 'label' in node:
            return 1
        return self._count_leaves(node['left']) + self._count_leaves(node['right'])


class PostPruneRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y, groups):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        if isinstance(groups, pd.Series):
            groups = groups.values

        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        self.trees = []
        for _ in range(self.n_estimators):
            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[idxs], y[idxs]
            group_sample = groups[idxs]

            tree = PostPruneDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X_sample, y_sample)
            tree.prune_for_fairness_and_accuracy(X_sample, y_sample, group_sample)
            self.trees.append(tree)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.round(tree_preds.mean(axis=0)).astype(int)

# Example usage
X = train['X']
y = train['y']
groups = np.where(train['p_attrs']["sex"].to_numpy() == "Male", 1, 0)

# Create and fit the model
post_prune_clf = PostPruneRandomForestClassifier(
    n_estimators=10,
    max_depth=50,
    random_state=42
)
post_prune_clf.fit(X, y, groups)

# Make predictions
predictions = post_prune_clf.predict(test['X'])


Initial number of leaves: 1030
Final number of leaves: 1021
Difference in number of leaves: 9
Fairness before pruning: 0.9928628374239481
Fairness after pruning: 0.9889863801514036
Difference in fairness: -0.003876457272544487
Accuracy before pruning: 0.9070285598541624
Accuracy after pruning: 0.9070285598541624
Difference in accuracy: 0.0
Initial number of leaves: 1021
Final number of leaves: 1010
Difference in number of leaves: 11
Fairness before pruning: 0.9950900841445636
Fairness after pruning: 0.9935181476749232
Difference in fairness: -0.0015719364696403781
Accuracy before pruning: 0.91452298966984
Accuracy after pruning: 0.91452298966984
Difference in accuracy: 0.0
Initial number of leaves: 1096
Final number of leaves: 1088
Difference in number of leaves: 8
Fairness before pruning: 0.9787652851153028
Fairness after pruning: 0.9808736165389893
Difference in fairness: 0.0021083314236864936
Accuracy before pruning: 0.9084464249544257
Accuracy after pruning: 0.9084464249544257
Diff

In [91]:
print("accuracy", np.count_nonzero(predictions == test['y'])/len(test['y']))
print("equal opportunity diff", equal_opportunity_diff(test['group_a'], test['group_b'], predictions, test['y']))
print("average odds diff", average_odds_diff(test['group_a'], test['group_b'], predictions, test['y']))

accuracy 0.6510121457489878
equal opportunity diff -0.04561899098027555
average odds diff -0.09601394830413507


### Baseline RandomForest Classifier

In [20]:
import numpy as np
from collections import Counter
import pandas as pd
from sklearn.metrics import recall_score, accuracy_score

class BaselineDecisionTreeClassifier:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        self.tree_ = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        if (depth >= self.max_depth or num_samples < self.min_samples_split or len(set(y)) == 1):
            return {'label': self._most_common_label(y)}

        best_split = self._find_best_split(X, y)
        if not best_split:
            return {'label': self._most_common_label(y)}

        left_idxs, right_idxs = self._split(X[:, best_split['feature_idx']], best_split['threshold'])

        left_subtree = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_subtree = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
                'left': left_subtree, 'right': right_subtree}

    def _find_best_split(self, X, y):
        num_samples, num_features = X.shape
        best_split = {}
        best_gain = -float('inf')

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                gain = self._calculate_information_gain(X, y, feature_idx, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_split = {'feature_idx': feature_idx, 'threshold': threshold}

        return best_split if best_gain != -float('inf') else None

    def _calculate_information_gain(self, X, y, feature_idx, threshold):
        left_idxs, right_idxs = self._split(X[:, feature_idx], threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        parent_entropy = self._entropy(y)
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, feature_column, threshold):
        left_idxs = np.argwhere(feature_column <= threshold).flatten()
        right_idxs = np.argwhere(feature_column > threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        return np.array([self._traverse_tree(x, self.tree_) for x in X])

    def _traverse_tree(self, x, node):
        if 'label' in node:
            return node['label']
        if x[node['feature_idx']] <= node['threshold']:
            return self._traverse_tree(x, node['left'])
        return self._traverse_tree(x, node['right'])


class BaselineRandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=5, min_samples_split=2, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        self.trees = []
        for _ in range(self.n_estimators):
            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[idxs], y[idxs]

            tree = BaselineDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.round(tree_preds.mean(axis=0)).astype(int)

# Example usage
X = train['X']
y = train['y']

# Create and fit the model
baseline_clf = BaselineRandomForestClassifier(
    n_estimators=3,
    max_depth=3,
    random_state=42
)
baseline_clf.fit(X, y)

# Make predictions
predictions = baseline_clf.predict(test['X'])

print("accuracy", np.count_nonzero(predictions == test['y'])/len(test['y']))
print("equal opportunity diff", equal_opportunity_diff(test['group_a'], test['group_b'], predictions, test['y']))
print("average odds diff", average_odds_diff(test['group_a'], test['group_b'], predictions, test['y']))


accuracy 0.6696356275303643
equal opportunity diff -0.03617801566062051
average odds diff -0.11421305731661568
